In [1]:
import numpy as np
import neurolab as nl
import os
from PIL import Image
import matplotlib.pyplot as plt

In [13]:
def CreateNeuralNetwork(nbInput:int, nbOutput:int, inputRange, nbLayers:int, nbNeuronsPerLayer):
    inputDescription = []
    for i in range(0, nbInputs):
        inputDescription.append(inputRange[i])

    layersDescription = []
    for i in range(0, nbLayers - 1):
        layersDescription.append(int(nbNeuronsPerLayers[i]))

    layersDescription.append(nbOutputs)
    return nl.net.newff(inputDescription, layersDescription)

#neural network Stats
imageSize = np.array([16, 16])
nbInputs = imageSize[0] * imageSize[1] * 3
nbOutputs = imageSize[0] * imageSize[1] * 3
nbLayers = 10
inputRange = [[0, 1] for i in range(0, nbInputs)]
nbNeuronsPerLayers = np.ones(nbLayers) * 15

net = CreateNeuralNetwork(nbInputs, nbOutputs, inputRange, nbLayers, nbNeuronsPerLayers)
print("nb Input : " + str(net.ci))
print("nb Output : " + str(net.co))
print("nb Layers : " + str(len(net.layers)))

nb Input : 768
nb Output : 768
nb Layers : 10


In [5]:
def Zeros(size:int):
    return [0 for i in range(0, size)]

class Dataset:
    def __init__(self, data, target):
        self.data = data
        self.target = target

def LoadDataSet(inputImagesDirectory:str, targetImagesDirectory:str,  nbMaxImages:int = -1):
    files = [f for f in os.listdir(inputImagesDirectory) if os.path.isfile(inputImagesDirectory + "/" + f) and f.endswith(".jpg")]
    if(nbMaxImages >= 1 and len(files) > nbMaxImages):
        files = files[0:nbMaxImages]
    images = [Image.open(inputImagesDirectory + imageName) for imageName in files]

    inputData = []
    for image in images:
        tmp = []
        for i in range(0, image.size[0]):
            for j in range(image.size[1]):
                pixel = image.getpixel((i, j))
                tmp.append(pixel[0] / 255)
                tmp.append(pixel[1] / 255)
                tmp.append(pixel[2] / 255)

        inputData.append(tmp)

    files = [f for f in os.listdir(targetImagesDirectory) if os.path.isfile(targetImagesDirectory + "/" + f) and f.endswith(".jpg")]
    if(nbMaxImages >= 1 and len(files) > nbMaxImages):
        files = files[0:nbMaxImages]
    images = [Image.open(inputImagesDirectory + imageName) for imageName in files]
    outputData = []
    for image in images:
        tmp = []
        for i in range(0, image.size[0]):
            for j in range(image.size[1]):
                pixel = image.getpixel((i, j))
                tmp.append(pixel[0] / 255)
                tmp.append(pixel[1] / 255)
                tmp.append(pixel[2] / 255)

        outputData.append(tmp)

    return Dataset(inputData, outputData)
    
dataSet = LoadDataSet("Cats/noisy_16/88%/", "Cats/pixellized_16/", 10)

In [6]:
def PlotError(err):
    plt.figure()
    plt.plot(err)
    plt.xlabel('Number of epochs')
    plt.ylabel('Training error')
    plt.grid()
    plt.show()

In [7]:
#test
#net pour trouver l'équation 2*x² + 8

min_val = -30
max_val = 30
num_points = 150
x = np.linspace(min_val, max_val, num_points)
y = 2 * np.square(x) + 8
y /= np.linalg.norm(y)

data = x.reshape(num_points, 1)
labels = y.reshape(num_points, 1)

plt.figure()
plt.scatter(data, labels)
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('Data-points')

#Now, build the neural network having two hidden layers with neurolab with ten neurons in the first hidden layer, six in the second hidden layer and one in the output layer.
neural_net = nl.net.newff([[min_val, max_val]], [10, 6, 1])

#using the gradient training algorithm −
neural_net.trainf = nl.train.train_gd

#train!
error = neural_net.train(data, labels, epochs = 1000, show = 100, goal = 0.01)

output = neural_net.sim(data)
y_pred = output.reshape(num_points)

plt.figure()
plt.plot(error)
plt.xlabel('Number of epochs')
plt.ylabel('Error')
plt.title('Training error progress')


x_dense = np.linspace(min_val, max_val, num_points * 2)
y_dense_pred = neural_net.sim(x_dense.reshape(x_dense.size,1)).reshape(x_dense.size)
plt.figure()
plt.plot(x_dense, y_dense_pred, '-', x, y, '.', x, y_pred, 'p')
plt.title('Actual vs predicted')
plt.show()


SyntaxError: 'return' outside function (3816193442.py, line 4)

In [12]:
def Train(dataset):
    return net.train(dataSet.data, dataSet.target,epochs = 500, show = 50, goal = 0.01)

"""
dataSet = LoadDataSet("Cats/noisy/88%/", "Cats/pixellized/", 10)

for i in range(0, len(dataSet.data)):
    dataSet.data[i] = dataSet.data[i][0 : nbEntriesAndOutput]
    dataSet.target[i] = dataSet.target[i][0 : nbEntriesAndOutput]
"""
err = Train(dataSet)

PlotError(err)

MemoryError: Unable to allocate 35.2 GiB for an array with shape (97167, 97167) and data type int32